In [1]:
!python -V

Python 3.13.5


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import pickle

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge

from sklearn.metrics import root_mean_squared_error

In [3]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2025/06/21 13:28:12 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/06/21 13:28:12 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/Users/wojciechskrzypinski/Documents/GitHub/mlops-zoomcamp/02-experiment_tracking/mlruns/1', creation_time=1750405740412, experiment_id='1', last_update_time=1750405740412, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
def read_dataframe(data):
    df = pd.read_parquet(data)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    numerical = ['trip_distance']
    
    df[categorical] = df[categorical].astype(str)

    return df

In [5]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [6]:
len(df_train), len(df_val)

(73908, 61921)

In [7]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [8]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [10]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715203628836

In [11]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [12]:
lr = Lasso(alpha=0.001)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

9.233436225720547

In [16]:
with mlflow.start_run():

    mlflow.set_tag("developer", "wojtek")
    mlflow.log_param("train-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)

    lr = Ridge()
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

In [10]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [11]:
train = xgb.DMatrix(X_train, label=y_train)
val = xgb.DMatrix(X_val, label=y_val)

In [25]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)

        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(val, 'validation')],
            early_stopping_rounds=50,
        )

        y_pred = booster.predict(val)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [26]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/opt/homebrew/Caskroom/miniconda/base/envs/mlops_env/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [10:19:00] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:11.27040                          
[1]	validation-rmse:10.47223                          
[2]	validation-rmse:9.80071                           
[3]	validation-rmse:9.23828                           
[4]	validation-rmse:8.77005                           
[5]	validation-rmse:8.38300                           
[6]	validation-rmse:8.06417                           
[7]	validation-rmse:7.80205                           
[8]	validation-rmse:7.58685                           
[9]	validation-rmse:7.41115                           
[10]	validation-rmse:7.26780                          
[11]	validation-rmse:7.15094                          
[12]	validation-rmse:7.05347                          
[13]	validation-rmse:6.97511                          
[14]	validation-rmse:6.91051                          
[15]	validation-rmse:6.85726                          
[16]	validation-rmse:6.81320                          
[17]	validation-rmse:6.77606                          
[18]	valid

/opt/homebrew/Caskroom/miniconda/base/envs/mlops_env/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [10:19:41] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:9.26165                                                    
[1]	validation-rmse:7.78894                                                    
[2]	validation-rmse:7.09693                                                    
[3]	validation-rmse:6.78175                                                    
[4]	validation-rmse:6.62972                                                    
[5]	validation-rmse:6.54851                                                    
[6]	validation-rmse:6.50075                                                    
[7]	validation-rmse:6.47450                                                    
[8]	validation-rmse:6.46024                                                    
[9]	validation-rmse:6.44432                                                    
[10]	validation-rmse:6.43005                                                   
[11]	validation-rmse:6.42439                                                   
[12]	validation-rmse:6.42071            

...

[3]	validation-rmse:6.77722                                                    
[4]	validation-rmse:6.74997                                                    
[5]	validation-rmse:6.73778                                                    
[6]	validation-rmse:6.73020                                                    
[7]	validation-rmse:6.71820                                                    
[8]	validation-rmse:6.71445                                                    
[9]	validation-rmse:6.71222                                                    
[10]	validation-rmse:6.70713                                                   
[11]	validation-rmse:6.70287                                                   
[12]	validation-rmse:6.69995                                                   
[13]	validation-rmse:6.69675                                                   
[14]	validation-rmse:6.69387                                                   
[15]	validation-rmse:6.68753            

/opt/homebrew/Caskroom/miniconda/base/envs/mlops_env/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [10:47:23] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[2]	validation-rmse:9.71463                                                    
[3]	validation-rmse:9.15076                                                    
[4]	validation-rmse:8.68952                                                    
[5]	validation-rmse:8.31417                                                    
[6]	validation-rmse:8.01034                                                    
[7]	validation-rmse:7.76492                                                    
[8]	validation-rmse:7.56794                                                    
[9]	validation-rmse:7.40926                                                    
[10]	validation-rmse:7.28115                                                   
[11]	validation-rmse:7.17848                                                   
[12]	validation-rmse:7.09643                                                   
[13]	validation-rmse:7.03067                                                   
[14]	validation-rmse:6.97751            

/opt/homebrew/Caskroom/miniconda/base/envs/mlops_env/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [10:48:06] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[1]	validation-rmse:7.95646                                                    
[2]	validation-rmse:7.27499                                                    
[3]	validation-rmse:6.95639                                                    
[4]	validation-rmse:6.79771                                                    
[5]	validation-rmse:6.71872                                                    
[6]	validation-rmse:6.67256                                                    
[7]	validation-rmse:6.64538                                                    
[8]	validation-rmse:6.62686                                                    
[9]	validation-rmse:6.61352                                                    
[10]	validation-rmse:6.60229                                                   
[11]	validation-rmse:6.59525                                                   
[12]	validation-rmse:6.59177                                                   
[13]	validation-rmse:6.58472            

In [29]:
params = {
    'learning_rate': 0.15390891484486,
    'max_depth': 30,
    'min_child_weight': 1.145405043044414,
    'objective': 'reg:linear',
    'reg_alpha': 0.1697761605394537,
    'reg_lambda': 0.011852507506273226,
    'seed': 42
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(val, 'validation')],
    early_stopping_rounds=50,
)

2025/06/21 10:56:34 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'cc9aa40bd6f047c894dfb52b703970da', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
/opt/homebrew/Caskroom/miniconda/base/envs/mlops_env/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [10:56:37] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:10.99145
[1]	validation-rmse:10.01437
[2]	validation-rmse:9.23965
[3]	validation-rmse:8.63315
[4]	validation-rmse:8.16241
[5]	validation-rmse:7.79778
[6]	validation-rmse:7.51885
[7]	validation-rmse:7.30148
[8]	validation-rmse:7.13548
[9]	validation-rmse:7.00732
[10]	validation-rmse:6.91018
[11]	validation-rmse:6.83496
[12]	validation-rmse:6.77643
[13]	validation-rmse:6.72781
[14]	validation-rmse:6.68973
[15]	validation-rmse:6.65765
[16]	validation-rmse:6.63309
[17]	validation-rmse:6.61292
[18]	validation-rmse:6.59504
[19]	validation-rmse:6.58292
[20]	validation-rmse:6.57064
[21]	validation-rmse:6.56142
[22]	validation-rmse:6.55271
[23]	validation-rmse:6.54609
[24]	validation-rmse:6.53922
[25]	validation-rmse:6.53391
[26]	validation-rmse:6.52982
[27]	validation-rmse:6.52701
[28]	validation-rmse:6.52505
[29]	validation-rmse:6.52263
[30]	validation-rmse:6.52109
[31]	validation-rmse:6.52028
[32]	validation-rmse:6.51862
[33]	validation-rmse:6.51703
[34]	validation-rmse:6

2025/06/21 10:57:13 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2025/06/21 10:57:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/21 10:57:13 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/homebrew/Caskroom/miniconda/base/envs/mlops_env/lib/python3.13/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [10:57:13] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/06/21 10:57:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [36]:
with mlflow.start_run():

    best_params = {
        'learning_rate': 0.15390891484486,
        'max_depth': 30,
        'min_child_weight': 1.145405043044414,
        'objective': 'reg:linear',
        'reg_alpha': 0.1697761605394537,
        'reg_lambda': 0.011852507506273226,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(val, 'validation')],
        early_stopping_rounds=50,
    )

    y_pred = booster.predict(val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    mlflow.log_artifact("models/lin_reg.bin", artifact_path="models_pickle")

    mlflow.xgboost.log_model(booster, artifact_path="xgboost_reg_model")

/opt/homebrew/Caskroom/miniconda/base/envs/mlops_env/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [11:49:26] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:10.99145
[1]	validation-rmse:10.01437
[2]	validation-rmse:9.23965
[3]	validation-rmse:8.63315
[4]	validation-rmse:8.16241
[5]	validation-rmse:7.79778
[6]	validation-rmse:7.51885
[7]	validation-rmse:7.30148
[8]	validation-rmse:7.13548
[9]	validation-rmse:7.00732
[10]	validation-rmse:6.91018
[11]	validation-rmse:6.83496
[12]	validation-rmse:6.77643
[13]	validation-rmse:6.72781
[14]	validation-rmse:6.68973
[15]	validation-rmse:6.65765
[16]	validation-rmse:6.63309
[17]	validation-rmse:6.61292
[18]	validation-rmse:6.59504
[19]	validation-rmse:6.58292
[20]	validation-rmse:6.57064
[21]	validation-rmse:6.56142
[22]	validation-rmse:6.55271
[23]	validation-rmse:6.54609
[24]	validation-rmse:6.53922
[25]	validation-rmse:6.53391
[26]	validation-rmse:6.52982
[27]	validation-rmse:6.52701
[28]	validation-rmse:6.52505
[29]	validation-rmse:6.52263
[30]	validation-rmse:6.52109
[31]	validation-rmse:6.52028
[32]	validation-rmse:6.51862
[33]	validation-rmse:6.51703
[34]	validation-rmse:6

2025/06/21 11:50:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/opt/homebrew/Caskroom/miniconda/base/envs/mlops_env/lib/python3.13/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [11:50:02] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/06/21 11:50:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [34]:
mlflow.xgboost.autolog(disable=True)

In [13]:
logged_model = 'runs:/f8a41854d0974847bc4587bb3377296f/xgboost_reg_model'

loaded_model = mlflow.pyfunc.load_model(logged_model)

In [14]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: /Users/wojciechskrzypinski/Documents/GitHub/mlops-zoomcamp/02-experiment_tracking/mlruns/1/models/m-b2c8708723fa4a8faadc2c693b62e0ae/artifacts
  flavor: mlflow.xgboost
  run_id: f8a41854d0974847bc4587bb3377296f

In [15]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

In [16]:
xgboost_model

In [17]:
y_pred = xgboost_model.predict(val)

In [18]:
y_pred[:10]

array([14.478492,  7.219713, 15.276713, 24.426598,  9.56972 , 17.15413 ,
       11.163117,  8.20489 ,  9.421057, 19.311848], dtype=float32)